## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
import numpy as np
from sklearn import datasets, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, RandomizedSearchCV

In [2]:
# 讀取資料集
wine = datasets.load_wine()

In [3]:
X = wine.data
Y = wine.target
Y_Name = wine.target_names
Feature_Name = wine.feature_names
print("Data Shape = ", X.shape)
print("Data Type = ", X.dtype)
print("Feature Shape = ", len(Feature_Name))
print("Kind of Targer = ", len(np.unique(Y)))

Data Shape =  (178, 13)
Data Type =  float64
Feature Shape =  13
Kind of Targer =  3


In [4]:
# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=4)

In [5]:
# 建立模型 (使用 5 顆樹，每棵樹的最大深度為 2)
clf = RandomForestClassifier(n_estimators=5, max_depth=2)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

acc = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)
print("Feature importance: ", clf.feature_importances_)

Accuracy:  0.9333333333333333
Feature importance:  [0.22139755 0.06706233 0.         0.0601883  0.         0.09730887
 0.2329408  0.01199835 0.         0.20631421 0.10278961 0.
 0.        ]


In [6]:
clf.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 2,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 5,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [7]:
# 設定要訓練的超參數組合
n_estimators = [int(x) for x in np.linspace(10, 2000, 20)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

param_grid = dict(n_estimators=n_estimators, max_features=max_features, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, bootstrap=bootstrap)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = RandomizedSearchCV(clf, param_grid, n_jobs=-1, verbose=1, cv = 10)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   41.2s finished
/Users/beemolin/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [8]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: 0.984962 using {'n_estimators': 1790, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 30, 'bootstrap': True}


In [9]:
# 使用最佳參數重新建立模型
clf_bestparam = RandomForestClassifier(n_estimators     =grid_result.best_params_['n_estimators'],
                                       min_samples_split=grid_result.best_params_['min_samples_split'],
                                       min_samples_leaf =grid_result.best_params_['min_samples_leaf'],
                                       max_features     =grid_result.best_params_['max_features'],
                                       max_depth        =grid_result.best_params_['max_depth'],
                                       bootstrap        =grid_result.best_params_['bootstrap'])
                                       
# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)
acc = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)
print("Feature importance: ", clf_bestparam.feature_importances_)

Accuracy:  0.9777777777777777
Feature importance:  [0.13712702 0.03097429 0.01174659 0.03624206 0.03129925 0.05106982
 0.13838127 0.00987943 0.02080291 0.16743849 0.07628636 0.11853625
 0.17021625]
